# Tesla 4680 model calibration

#### Activate environment

In [2]:
using Pkg
Pkg.activate("../../../battmo_env")


  Activating project at `~/repositories/battmo_env`


#### Import packages

In [3]:
using BattMo, GLMakie, CSV, DataFrames, Jutul, Statistics


SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up

SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up

SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up


### Create cell parameter set

In [4]:
model_settings = load_model_settings(;from_default_set = "p4d_cylindrical")
print_info(model_settings)


PARAMETER OVERVIEW
Parameter                                                                                 Unit                Type                Value                         
----------------------------------------------------------------------------------------------------------------------------------------------------------------
[ "ButlerVolmer" ]                                                                        N/A                 String              Standard                      
[ "CurrentCollectors" ]                                                                   N/A                 String              Standard                      
[ "ModelFramework" ]                                                                      N/A                 String              P4D Cylindrical               
[ "PotentialFlowDiscretization" ]                                                         N/A                 String              GeneralAD                     
[ "RampUp" ]  

In [5]:
model = LithiumIonBattery(;model_settings)
cell_parameters = load_cell_parameters(;from_model_template = model)
print_info(cell_parameters)

✔️ Validation of ModelSettings passed: No issues found.
──────────────────────────────────────────────────

PARAMETER OVERVIEW
Parameter                                                                                 Unit                Type                Value                         
----------------------------------------------------------------------------------------------------------------------------------------------------------------
[ "Cell" ][ "InnerRadius" ]                                                               m                   Float64             0.0                           
[ "Cell" ][ "OuterRadius" ]                                                               m                   Float64             0.0                           
[ "Electrolyte" ][ "ChargeNumber" ]                                                       1                   Float64             0.0                           
[ "Electrolyte" ][ "Concentration" ]                                

The paper provides the following experimental data that we can use to create the cell parameter set that we need:

**Cell properties**
- NominalVoltage = 3.7 V
- NominalCapacity = 22 Ah
- Mass = 355.9 g
- EnergyDensity = 622.4 Wh/L
- Specific Energy = 232.5 Wh/kg
- OuterRadius = 22.25 mm
- InnerRadius = 2.5 mm
- Height = 71 mm
- Volume = 125.6 cm^3
- ElectrodeLength = 3403 mm - 136 mm
- NumberOfSeparators = 2
- DoubleSidedElectrodes

**Electrode coating properties**
- Thickness: measurements along the electrode length
- Density: measurements along the electrode length
- Mass: measurements along the electrode length

**Positive electrode active material**
- Material = NMC811
- ParticleRadius = between 3 and 16 microm

**Negative electrode active material**
- Material = Graphite
- ParticleRadius = up to 16 microm

**Positive electrode binder**
- Material = PVdF (polyvinylidene fluoride)

**Negative electrode binder**
- Material = PTFE (polytetrafluoroethylene)

**Negative electrode additive**
- Material = Carbon Black

**Positive electrode current collector properties**
- Material = Copper
- Thickness = 4,5 microm

**Negative electrode current collector properties**
- Material = Aluminium
- Thickness = 4,5 microm

**Separator properties**
- Thickness = 71 mm

**Electrolyte properties**
- Material = LiPF6 (lithium hexafluorophosphate) or LiTFSI (lithium bis(fluorosulfonyl)imide)

**Galvanostatic EIS measurement**
For full cell, positive and negative electrode

**pOCV measurements**
- Full cell, negative and positive electrode voltage curves with a current of C/50
- full cell C/20 pOCV of one of the cells

**DVA and ICA data**
From one of the cells

**HPPC measurements**
From one of the cells

##### What is a pseudo OCV curve?
The measured voltage of a full cell or one of the electrodes for a c-rate low enough so that the curve represents the open circuit voltage. At low c-rate the diffusion and kinetics have negligible over potentials.

##### What is a galvanostatic EIS measurement?
EIS stands for Electrochemical Impedance Spectroscopy

##### What is DVA data?

##### What is ICA data?

##### What is HPPC data?

##### Cel properties

In [7]:
cell_parameters["Cell"]

Dict{Any, Any} with 2 entries:
  "InnerRadius" => 0.0
  "OuterRadius" => 0.0

In [14]:
cell_parameters["Cell"]["InnerRadius"] = 2.5e-3
cell_parameters["Cell"]["OuterRadius"] = 22.25e-3
cell_parameters["Cell"]["Height"] = 71e-3

electrode_length = 3403e-3 - 136e-3
cell_parameters["Cell"]["ElectrodeGeometricSurfaceArea"] = electrode_length*cell_parameters["Cell"]["Height"] 

0.23195699999999997

We can add additional information of the cell. For example, cell KPIs that we can compare our model to later on.

In [18]:
print_info("EffectiveDensity")


----------------------------------------------------------------------------------------------------
🔋  Cell Parameter:  EffectiveDensity
----------------------------------------------------------------------------------------------------
    🔹 Name               EffectiveDensity
    🔹 Category           CellParameters
    🔹 Description        Density of a porous electrode, calculated from its geometrical volume and mass. It includes the volume occupied by pores.
    🔹 Type               Real
    🔹 Unit               kg·m⁻³
    🔹 Minimum value      0.0
    🔹 Maximum value      23000.0



In [17]:
cell_parameters["Cell"]["NominalVoltage"] = 3.7
cell_parameters["Cell"]["NominalCapacity"] = 22
cell_parameters["Cell"]["Mass"] = 355.9
cell_parameters["Cell"]["Volume"] = 125.6e-6
cell_parameters["Cell"]["EnergyDensity"] = 622.4
cell_parameters["Cell"]["SpecificEnergy"] = 232.5
print_info(cell_parameters)



PARAMETER OVERVIEW
Parameter                                                                                 Unit                Type                Value                         
----------------------------------------------------------------------------------------------------------------------------------------------------------------
[ "Cell" ][ "ElectrodeGeometricSurfaceArea" ]                                             m²                  Float64             0.23196                       
[ "Cell" ][ "EnergyDensity" ]                                                             N/A                 Float64             622.4                         
[ "Cell" ][ "Height" ]                                                                    m                   Float64             0.071                         
[ "Cell" ][ "InnerRadius" ]                                                               m                   Float64             0.0025                        
[ "Cell" ][ "M

##### Electrode properties

In [21]:
# Read the electrode properties into a DataFrame
df_electrodes = CSV.read(joinpath(dirname(@__DIR__),"example_input","cell_parameters", "ank_2023", "Electrode_properties.txt"), DataFrame)

Row,Length cathode,Mass cathode,Thickness cathode,Density cathode,Length anode,Mass anode,Thickness anode,Density anode
,Float64,Float64,Int64,Float64,Float64,Float64,Int64,Float64
1,0.0,26.6708,175,3.31314,0.0,19.6598,269,1.51813
2,0.5,26.4466,174,3.30582,0.5,19.456,258,1.56904
3,1.0,26.4563,172,3.3489,1.0,19.1929,255,1.56677
4,1.5,26.6448,173,3.35154,1.5,19.5381,258,1.57565
5,2.0,26.557,173,3.34051,2.0,19.7758,265,1.55104
6,2.5,26.557,174,3.31963,2.5,19.4221,266,1.51735
7,3.0,26.7227,176,3.2991,3.0,19.7107,265,1.54594
8,3.37,26.6025,178,3.24421,3.38,19.3966,268,1.50361


In [22]:
# Calculate average properties
averages = Dict(
	"NegativeElectrodeCoatingThickness" => mean(df_electrodes[!, "Thickness anode"])/2,
	"PositiveElectrodeCoatingThickness" => mean(df_electrodes[!, "Thickness cathode"])/2,
	"NegativeElectrodeCoatingMass" => mean(df_electrodes[!, "Mass anode"])/2,
	"PositiveElectrodeCoatingMass" => mean(df_electrodes[!, "Mass cathode"])/2,
	"NegativeElectrodeCoatingEffectiveDensity" => mean(df_electrodes[!, "Density anode"])*1000, # convert from g/cm3 to kg/m3
	"PositiveElectrodeCoatingEffectiveDensity" => mean(df_electrodes[!, "Density cathode"])*1000, # convert from g/cm3 to kg/m3
)

Dict{String, Float64} with 6 entries:
  "NegativeElectrodeCoatingThickness"        => 131.5
  "NegativeElectrodeCoatingEffectiveDensity" => 1543.44
  "PositiveElectrodeCoatingThickness"        => 87.1875
  "PositiveElectrodeCoatingEffectiveDensity" => 3315.36
  "NegativeElectrodeCoatingMass"             => 9.7595
  "PositiveElectrodeCoatingMass"             => 13.2911

In [23]:
cell_parameters["NegativeElectrode"]["Coating"]["Thickness"] = averages["NegativeElectrodeCoatingThickness"]
cell_parameters["NegativeElectrode"]["Coating"]["EffectiveDensity"] = averages["NegativeElectrodeCoatingEffectiveDensity"]
cell_parameters["PositiveElectrode"]["Coating"]["Thickness"] = averages["PositiveElectrodeCoatingThickness"]
cell_parameters["PositiveElectrode"]["Coating"]["EffectiveDensity"] = averages["PositiveElectrodeCoatingEffectiveDensity"]
print_info(cell_parameters)


PARAMETER OVERVIEW
Parameter                                                                                 Unit                Type                Value                         
----------------------------------------------------------------------------------------------------------------------------------------------------------------
[ "Cell" ][ "ElectrodeGeometricSurfaceArea" ]                                             m²                  Float64             0.23196                       
[ "Cell" ][ "EnergyDensity" ]                                                             N/A                 Float64             622.4                         
[ "Cell" ][ "Height" ]                                                                    m                   Float64             0.071                         
[ "Cell" ][ "InnerRadius" ]                                                               m                   Float64             0.0025                        
[ "Cell" ][ "M

#### Load parameters and settings

In [8]:
cell_parameters = load_cell_parameters(from_file_path = joinpath(dirname(@__DIR__), "example_input", "cell_parameters", "tesla_4680_before_calibration.json"))
cycling_protocol = load_cycling_protocol(from_default_set = "cc_discharge")
model_settings = load_model_settings(from_default_set = "p2d")
solver_settings = load_solver_settings(from_default_set = "direct");

In [9]:
quick_cell_check(cell_parameters)

════════════════════════════════════════════════════════════════════════════════════════════════════════════
🔋 Quick Cell Check
════════════════════════════════════════════════════════════════════════════════════════════════════════════
Cell 1: Ank2023

Quantity                                Cell 1         | Unit         | Source
────────────────────────────────────────────────────────────────────────────────────────────────────────────
Nominal Voltage                         0              | V            | [INPUT]     
Nominal Capacity                        22.65          | Ah           | [INPUT]     
Cell Theoretical Capacity               0.02416        | Ah           | [EQUILIBRIUM CALCULATION]
Cell N:P Ratio                          97.37          | -            | [EQUILIBRIUM CALCULATION]
Cell Mass                               0.28           | kg           | [EQUILIBRIUM CALCULATION]
Positive Electrode Mass Loading         27.6           | mg/cm²       | [EQUILIBRIUM CALCULATI

#### Alter cycling protocol

In [10]:
# cycling_protocol["InitialTemperature"] = 298.15
cycling_protocol["UpperVoltageLimit"] = 3.7
cycling_protocol["LowerVoltageLimit"] = 3.0
cycling_protocol["DRate"] = 1

1

#### Run simulation

In [11]:
model = LithiumIonBattery(; model_settings)

sim = Simulation(model, cell_parameters, cycling_protocol)

output = solve(sim; solver_settings, accept_invalid = true);

✔️ Validation of ModelSettings passed: No issues found.
──────────────────────────────────────────────────
✔️ Validation of CellParameters passed: No issues found.
──────────────────────────────────────────────────
✔️ Validation of CyclingProtocol passed: No issues found.
──────────────────────────────────────────────────
✔️ Validation of SimulationSettings passed: No issues found.
──────────────────────────────────────────────────
✔️ Validation of SolverSettings passed: No issues found.
──────────────────────────────────────────────────
Jutul: Simulating 1 hour, 6 minutes as 84 report steps


Progress   2%|█                                          |  ETA: 0:11:07

Progress 100%|███████████████████████████████████████████| Time: 0:00:18
   Progress: Solved step 84/84
      Stats: 179 iterations in 16.94 s (94.66 ms each)


╭────────────────┬──────────┬──────────────┬──────────╮
│ Iteration type │ Avg/step │ Avg/ministep │    Total │
│                │ 84 steps │ 84 ministeps │ (wasted) │
├────────────────┼──────────┼──────────────┼──────────┤
│ Newton         │  2.13095 │      2.13095 │  179 (0) │
│ Linearization  │  3.13095 │      3.13095 │  263 (0) │
│ Linear solver  │  2.13095 │      2.13095 │  179 (0) │
│ Precond apply  │      0.0 │          0.0 │    0 (0) │
╰────────────────┴──────────┴──────────────┴──────────╯
╭───────────────┬─────────┬────────────┬─────────╮
│ Timing type   │    Each │   Relative │   Total │
│               │      ms │ Percentage │       s │
├───────────────┼─────────┼────────────┼─────────┤
│ Properties    │  0.9336 │     0.99 % │  0.1671 │
│ Equations     │ 33.4868 │    51.98 % │  8.8070 │
│ Assembly      │  6.5071 │    10.10 % │  1.7114 │
│ Linear solve  │  1.5193 │     1.61 % │  0.2720 │
│ Linear setup  │  0.0000 │     0.00 % │  0.0000 │
│ Precond apply │  0.0000 │     0.00 

#### Plot results

In [12]:
plot_dashboard(output; plot_type = "contour")